First, load the 311 complaints dataset available here https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [1]:
CSV_PATH = './data/311.csv'
import subprocess
from tqdm import tqdm
import pandas as pd 
import os
from h3 import h3
import warnings
warnings.filterwarnings('ignore')


def file_len(fname):
    p = subprocess.Popen(['wc', '-l', fname], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result, err = p.communicate()
    if p.returncode != 0:
        raise IOError(err)
    return int(result.strip().split()[0])+1

n_rows = file_len(CSV_PATH)
print (f'Exact number of rows: {n_rows}')
df_tmp = pd.read_csv(CSV_PATH, nrows=5)
df_tmp.head()


OSError: b'wc: ./311.csv: open: No such file or directory\n'

In [ ]:
types = {
    #'Unique Key': 'float32',
              'Complaint Type': 'category', 
              'Longitude': 'float32',
              'Latitude': 'float32',
#              'Incident Zip': 'int',
#              'Created Date': 'str',
#              'Closed Date': 'str',

        }
cols = list(types.keys())
chunksize = 5_000_000

df_list = [] # list to hold the batch dataframe

def process_date(df_chunk, key):
    df_chunk[key] = df_chunk[key].str.slice(0, 16)
    #df_chunk[key] = pd.to_datetime(df_chunk[key], utc=True, format='%Y-%m-%d %H:%M')
    
for df_chunk in tqdm(pd.read_csv(CSV_PATH, usecols=cols, dtype=types, chunksize=chunksize)):
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
#    process_date(df_chunk, 'Closed Date')
#    process_date(df_chunk, 'Created Date')
    df_list.append(df_chunk) 

    
pdf = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
pdf.info()
#takes 1:30 minutes

In [ ]:
pd.options.display.max_rows = 2000

counts = pdf['Complaint Type'].value_counts()
print(counts)



In [ ]:
APERTURE_SIZE = 9
hex_col = 'hex'+str(APERTURE_SIZE)


pdf[hex_col] = 0
import json

print(1)
def binRow(df311):
    # find hexs containing the points
    df311[hex_col] = df311.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,APERTURE_SIZE),1)

    # aggregate the points
    df311g = df311.groupby(hex_col).size()#.to_frame('cnt').reset_index()

    #find center of hex for visualization
#     df311g['lat'] = df311g[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
#     df311g['lng'] = df311g[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
    return (df311g)


def saveComplaint(complaint):
    print(complaint)
    data = pdf.loc[pdf['Complaint Type'] ==(complaint)]
    binned = binRow(data)
    complaint = complaint.replace(' ', '-').replace('/','-')
    hi = [(hex, count)  for hex,count in binned.iteritems()]
    with open('./data/'+complaint+'.json', 'w') as outfile: json.dump(hi, outfile)
    
stuff = ['Noise - Residential',                         
'HEAT/HOT WATER' ,                              
'Street Condition'   ,                           
'Illegal Parking ',                              
'Blocked Driveway'  ,                            
'Street Light Condition' ,                       
'HEATING'          ,                             
'PLUMBING'     ,                                 
'Water System'  ,                                
'Noise - Street/Sidewalk' ,                      
'GENERAL CONSTRUCTION'  ,                                                                
'UNSANITARY CONDITION']                          
for complaint in stuff:
    saveComplaint(complaint)



In [ ]:
# data = pdf.loc[pdf['Complaint Type'] ==('Noise - Residential')]
# binned = binRow(data)
for hex, row in binned.iteritems():
    print(hex, row)

In [ ]:
#automatically upload all csv to s3
import boto3, os